In [3]:
import pandas as pd
import fx_data_cleaner

import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

imported successfully


In [4]:
# preliminary data cleaning
usd_data = fx_data_cleaner.csv_to_columns('usdcad h1 2008 to 2024.csv')
eur_data = fx_data_cleaner.csv_to_columns('eurcad h1 2008 to 2024.csv')

# load analytical base tables

usdcad_h1 = pd.DataFrame(usd_data)
eurcad_h1 = pd.DataFrame(eur_data)

print(usdcad_h1.tail(10))
print(eurcad_h1.tail(10))

              date      time     open     high      low    close tick_vol vol  \
100001  2024.07.29  01:00:00  1.38322  1.38336  1.38250  1.38311      563   0   
100002  2024.07.29  02:00:00  1.38311  1.38323  1.38245  1.38304      670   0   
100003  2024.07.29  03:00:00  1.38304  1.38316  1.38161  1.38188     1287   0   
100004  2024.07.29  04:00:00  1.38188  1.38244  1.38176  1.38191     1675   0   
100005  2024.07.29  05:00:00  1.38192  1.38216  1.38177  1.38205     1351   0   
100006  2024.07.29  06:00:00  1.38206  1.38223  1.38192  1.38195      711   0   
100007  2024.07.29  07:00:00  1.38194  1.38267  1.38186  1.38259      618   0   
100008  2024.07.29  08:00:00  1.38258  1.38262  1.38219  1.38257      839   0   
100009  2024.07.29  09:00:00  1.38257  1.38265  1.38201  1.38249     1102   0   
100010  2024.07.29  10:00:00  1.38249  1.38385  1.38230  1.38375     1567   0   

       spread  
100001      5  
100002      5  
100003      5  
100004      4  
100005      4  
100006      

In [5]:
# amend datatypes for each column. combine date and time to datetime
usdcad_h1 = fx_data_cleaner.fx_data_type(usdcad_h1)
eurcad_h1 = fx_data_cleaner.fx_data_type(eurcad_h1)

print(usdcad_h1.info())
print(eurcad_h1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   open       100000 non-null  float64       
 1   high       100000 non-null  float64       
 2   low        100000 non-null  float64       
 3   close      100000 non-null  float64       
 4   tick_vol   100000 non-null  int32         
 5   vol        100000 non-null  int64         
 6   spread     100000 non-null  int32         
 7   date_time  100000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int32(2), int64(1)
memory usage: 6.1 MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 100900 entries, 0 to 100899
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   open       100900 non-null  float64       
 1   high       100900 non-null  float64       
 2   low        100900 non-null  fl

In [6]:
print(usdcad_h1.describe().T)
print(eurcad_h1.describe().T)

              count                           mean                  min  \
open       100000.0                       1.206908               0.9414   
high       100000.0                        1.20781              0.94207   
low        100000.0                       1.206015              0.94062   
close      100000.0                       1.206915              0.94141   
tick_vol   100000.0                     2725.73743                  1.0   
vol        100000.0                593465498.36546                  0.0   
spread     100000.0                        9.98687                  0.0   
date_time    100000  2016-07-09 10:36:52.848000256  2008-06-11 08:00:00   

                           25%                  50%                  75%  \
open                  1.053127              1.26213              1.32397   
high                   1.05428              1.26308              1.32475   
low                    1.05222              1.26112              1.32323   
close               

In [7]:
print(usdcad_h1.tail(10))
print(eurcad_h1.tail(10))

          open     high      low    close  tick_vol  vol  spread  \
99990  1.38135  1.38192  1.38134  1.38148      1026    0       4   
99991  1.38149  1.38312  1.38144  1.38221      2133    0       2   
99992  1.38220  1.38226  1.38070  1.38142      2577    0       2   
99993  1.38145  1.38448  1.38102  1.38383      3579    0       4   
99994  1.38384  1.38487  1.38335  1.38337      2630    0       4   
99995  1.38337  1.38366  1.38214  1.38220      1842    0       2   
99996  1.38219  1.38365  1.38196  1.38332      2192    0       2   
99997  1.38332  1.38395  1.38292  1.38376      1914    0       4   
99998  1.38376  1.38398  1.38306  1.38308      1436    0       4   
99999  1.38310  1.38373  1.38286  1.38368       660    0       4   

                date_time  
99990 2024-07-26 14:00:00  
99991 2024-07-26 15:00:00  
99992 2024-07-26 16:00:00  
99993 2024-07-26 17:00:00  
99994 2024-07-26 18:00:00  
99995 2024-07-26 19:00:00  
99996 2024-07-26 20:00:00  
99997 2024-07-26 21:00:00  

In [8]:

if 'date_time' in usdcad_h1.columns:
    usdcad_h1.index = usdcad_h1['date_time']
    usdcad_h1.drop(columns='date_time', inplace=True)
else: pass
print(usdcad_h1.head())


if 'date_time' in eurcad_h1.columns:
    eurcad_h1.index = eurcad_h1['date_time']
    eurcad_h1.drop(columns='date_time', inplace=True)
else:
    pass
print(eurcad_h1.head())

                       open    high     low   close  tick_vol  vol  spread
date_time                                                                 
2008-06-11 08:00:00  1.0237  1.0239  1.0213  1.0217       403    0      30
2008-06-11 09:00:00  1.0219  1.0227  1.0214  1.0220       391    0      30
2008-06-11 10:00:00  1.0221  1.0222  1.0196  1.0196       399    0      30
2008-06-11 11:00:00  1.0197  1.0205  1.0173  1.0192       480    0      30
2008-06-11 12:00:00  1.0193  1.0195  1.0184  1.0189       239    0      30
                       open    high     low   close  tick_vol  vol  spread
date_time                                                                 
2008-04-18 00:00:00  1.6061  1.6078  1.6060  1.6064       166    0      80
2008-04-18 01:00:00  1.6065  1.6076  1.6061  1.6072       303    0      80
2008-04-18 02:00:00  1.6071  1.6072  1.6012  1.6021       587    0      80
2008-04-18 03:00:00  1.6019  1.6038  1.6010  1.6033       358    0      80
2008-04-18 04:00:00  1.60

In [9]:
y_usd = usdcad_h1['close']
y_usd.index = usdcad_h1.index
usd_train, usd_test = np.split(y_usd, [int(0.85 * len(usdcad_h1))])

print('usdcad training data:', usd_train.shape)
print('usdcad testing data:', usd_test.shape)

y_eur = eurcad_h1['close']
y_eur.index = eurcad_h1.index
eur_train, eur_test = np.split(y_eur, [int(0.85 * len(eurcad_h1))])

print('eurcad training data:', eur_train.shape)
print('eurcad testing data:', eur_test.shape)

usdcad training data: (85000,)
usdcad testing data: (15000,)
eurcad training data: (85765,)
eurcad testing data: (15135,)


In [10]:
usdcad_h1_model = ARIMA(usd_train)
usdcad_h1_model_fit = usdcad_h1_model.fit()
usdcad_h1_pred = usdcad_h1_model_fit.forecast(steps = len(usd_test))
usdcad_mse = mean_squared_error(np.array(usd_test), usdcad_h1_pred)
print('Mean Squared Error:', usdcad_mse)

Mean Squared Error: 0.024785463700694132


In [11]:
eurcad_h1_model = ARIMA(eur_train)
eurcad_h1_model_fit = eurcad_h1_model.fit()
eurcad_h1_pred = eurcad_h1_model_fit.forecast(steps=len(eur_test))
eurcad_mse = mean_squared_error(np.array(eur_test), eurcad_h1_pred)
print('Mean Squared Error:', eurcad_mse)

Mean Squared Error: 0.004010391789780199


In [12]:
import numpy as np
np.sqrt(usdcad_mse)

0.15743399791879178